# 聚类网络攻击

In [1]:
import scapy.all as scapy
import os
import numpy as np
import hashlib
from sklearn.cluster import DBSCAN,KMeans
from matplotlib import pyplot as plt

In [2]:
data_path = 'data/0.pcap'

## 观看数据

In [3]:
view_p = scapy.rdpcap(data_path)
view_p[5].show()

###[ Ethernet ]### 
  dst       = 00:50:56:bd:10:ba
  src       = 00:50:56:bd:4a:f1
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 1420
     id        = 40586
     flags     = DF
     frag      = 0
     ttl       = 64
     proto     = tcp
     chksum    = 0x71ae
     src       = 198.19.20.5
     dst       = 198.18.133.8
     \options   \
###[ TCP ]### 
        sport     = http
        dport     = 58722
        seq       = 3119809356
        ack       = 425179630
        dataofs   = 8
        reserved  = 0
        flags     = PA
        window    = 235
        chksum    = 0x91fd
        urgptr    = 0
        options   = [('NOP', None), ('NOP', None), ('Timestamp', (4820152, 949618283))]
###[ Raw ]### 
           load      = 'n\xbb\x9cm\xc5L\xec\x02\xac\xd6\xcf.#]\xbcw\xa5x\xad\n\x96#\xe6\x06\x85\x82\x06\'p\xc1\xa3\xc9\x88\xe5?\xadM\xd9\xa2\x8d\xf7 {[w\x0b \x1c\x1e\x05B\xd1[\x96\xc2/\xd6\xa7\x10\xaa\x94\x19\xf9I\'\x14\xd1\x11,

## 数据载入与处理

In [4]:
def make_data(data_path):
    datas = []
    data_index = []
    sha256 = hashlib.sha256()
    packets = scapy.rdpcap(data_path)
    for i,packet in enumerate(packets):
        this_data = []
        try:
            sha256.update(packet.load)
            payload = sha256.hexdigest()
            for char in payload:
                this_data.append(int(char,16)/15.0)
            data = np.array(this_data)
            datas.append(data)
            data_index.append(i)
        except:
            continue
        
           
    return np.array(datas),data_index

datas,data_index = make_data(data_path)
print("数据维度：{}".format(datas.shape))

数据维度：(366, 64)


In [5]:
# 攻击包
att_data = 'data/1、防御直接攻击-ACL and no-IPS-01.pcap'
datas,data_index = make_data(att_data)
print("数据维度：{}".format(datas.shape))

数据维度：(12, 64)


## 模型训练

In [6]:
# k-mean 模型
kmean_model = KMeans(5).fit(datas)
kmean_labels = kmean_model.labels_
print(kmean_labels)

[3 0 0 4 2 2 4 4 1 1 3 2]


In [17]:
# DBSCAN 模型
db_model = DBSCAN(eps=3.1, min_samples=2).fit(datas)
db_labels = db_model.labels_
print(db_labels)

[ 0 -1 -1 -1 -1  1  2  2  1 -1  0 -1]


In [18]:
data_index

[3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 21]

In [19]:
view_p = scapy.rdpcap(att_data)
view_p[10].load

b'ker tag - Pick a time</a></li>\n\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t</ul>\n\t\t\t\t\t\t\t</li>\n\t\t\t\t\t\t</ul>\n\t\t\t\t\t</li>\n\t\t\t\t\t<li class="dropdown">\n\t\t\t\t\t\t<a href="#" class="dropdown-toggle" data-toggle="dropdown">File<b class="caret"></b></a>\n\t\t\t\t\t\t<ul class="dropdown-menu">\n\t\t\t\t\t\t\t<li><a href="/struts2-showcase/filedownload/index.action;jsessionid=E526E0E3CAC63D2DCFFC0F7D154BA001">File Download</a></li>\n\t\t\t\t\t\t\t<li class="dropdown-submenu">\n\t\t\t\t\t\t\t\t<a href="#">File Upload</a>\n\t\t\t\t\t\t\t\t<ul class="dropdown-menu">\n\t\t\t\t\t\t\t\t\t<li>\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t<a href="/struts2-showcase/fileupload/upload.action;jsessionid=E526E0E3CAC63D2DCFFC0F7D154BA001">Single File Upload</a>\n\t\t\t\t\t\t\t\t\t</li>\n\t\t\t\t\t\t\t\t\t<li>\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t<a href="/struts2-showcase/fileupload/multipleUploadUsingList.action;jsessionid=E526E0E3CAC63D2DCFFC0F7D154BA001">Multiple File Upload (Lis

In [20]:
view_p[11].load

b'down-menu">\n\t\t\t\t\t\t\t\t\t<li><a href="/struts2-showcase/hangman/hangmanNonAjax.action;jsessionid=E526E0E3CAC63D2DCFFC0F7D154BA001">Hangman (Non Ajax)</a></li>\n\t\t\t\t\t\t\t\t\t<li><a href="/struts2-showcase/hangman/hangmanAjax.action;j\r\n1355\r\nsessionid=E526E0E3CAC63D2DCFFC0F7D154BA001">Hangman (Ajax - Experimental)</a></li>\n\t\t\t\t\t\t\t\t</ul>\n\t\t\t\t\t\t\t</li>\n\t\t\t\t\t\t\t<li><a href="/struts2-showcase/person/index.html;jsessionid=E526E0E3CAC63D2DCFFC0F7D154BA001">Person Manager</a></li>\n\t\t\t\t\t\t\t<li><a href="/struts2-showcase/skill/index.html;jsessionid=E526E0E3CAC63D2DCFFC0F7D154BA001">CRUD</a></li>\n\t\t\t\t\t\t\t<li><a href="/struts2-showcase/wait/index.html;jsessionid=E526E0E3CAC63D2DCFFC0F7D154BA001">Execute &amp; Wait</a></li>\n\t\t\t\t\t\t\t<li><a href="/struts2-showcase/token/index.html;jsessionid=E526E0E3CAC63D2DCFFC0F7D154BA001">Token</a></li>\n\t\t\t\t\t\t\t<li><a href="/struts2-showcase/validation/index.action;jsessionid=E526E0E3CAC63D2DCFFC0F